In this exercise, you will code a neural network for classifying audio segements for five different syllables: 'a' (as in 'bar'), 'e' (as in 'bed'), 'i' (as in 'big'), 'o' (as in 'shop'), 'u' (as in 'look').

Download and unzip the data from [here](https://drive.google.com/open?id=1mzoJ-l99yduGDs4we3iL1D3qssHFjbne) and listen to the different .wav files. 

As features, we will use the spectrograms extracted from each wav file, that look something like the image [here](https://drive.google.com/open?id=19vXRpeeNHt2wv0e9o3KvEKYjxQzqa1WK). The spectrogram is acquired by applying a Fourier transform on overlapping 25ms windows on the audio. The x-axis is the time, and the y-axis is the frequency.

To upload the data into this notebook, follow the instructions of the **"Prepare the data"**-section below.

To visualize more spectrograms, use the **"Visualize spectrogram"** cell below.

Each wav file is one second, and the corresponding spectrogram is of shape \[97, 201] (97 time steps, 201 frequencies). You don't have to know much about the Fourier transform and audio/signal processing in order to classify audio segments with a neural network. From now on, we can just assume every audio segment is represented by a sequence of 97 time steps, where at each time step we have 201 features, regardless of their meaning. 

Our model will use a simple recurrent layer / LSTM layer. To compute the logits, we will apply an additional fully connected layer on the state of the last time step. On the logits we will apply `softmax` and compute the `cross entropy loss`. 

We will be implementing our custom RNN cells, which are fed in a custom RNN Layer, already implemented below.

## Initial imports

In [ ]:
import os
import torch
import numpy as np
import typing
import numpy as np
from matplotlib import pyplot as plt

print(torch.__version__)

## Prepare the data

1. Download the data and load it into the memory. Follow either the steps for your local python/jupyter notebook installation or the steps for google colab.

### Steps for your local jupyter notebook

If you are using jupyter notebook on your local machine, download the vowels.zip file, extract it, and set the path of the 'basefolder' variable to the vowels-directory. E.g. basefolder = "home/user/DeepLearningPraktikum/Exercise4/vowels/"

In [ ]:
basefolder = "" #TODO Where you extracted the data, leave empty in Colab

### Steps for google colab

If you are using google colab there are a few possibilities for you to upload the vowels-data into the colab notebook.


#### Possibility 1

Use the code in the cell below to load the vowels.zip file into your colab-notebook.

In [ ]:
!wget -O vowels.zip "https://drive.google.com/uc?export=download&id=1mzoJ-l99yduGDs4we3iL1D3qssHFjbne" && unzip vowels.zip -d vowels && rm vowels.zip

basefolder = "vowels"

#### Possibility 2
1. Upload the extracted vowels folder to your google drive.
2. Mount your drive to collaboratory by using the cell below
3. Now, set the basefolder-variable to the path in your drive where the vowels-folder is located. E.g. basefolder = "/content/drive/MyDrive/vowels" or something like basefolder = "/content/vowels". One way to find out the path to the vowel-folder is by having a look at the files in the panel on the left side of your notebook.

In [ ]:
# Mount your google drive 
from google.colab import drive
drive.mount('/content/drive')

# Set the basefolder-path
basefolder = "" #TODO Where your data is located in your drive, leave empty if you are using your local jupyter notebook or already did option 1.

## Start Exercises

2. Get acquainted to the data by listening to the audio files, looking at the spectrogram depicted below and by investigating the code for data preparation below

## Visualize spectrogram

In [ ]:
import os
import torch
import numpy as np
import typing
from matplotlib import pyplot as plt
# basefolder = "~/vowels/"

audio = 'a/8.npy'
mag = np.load(os.path.join(basefolder, audio))
mag = 20.*np.log10(np.abs(mag)/10e-6) # amplitude to decibel

plt.figure(figsize=(15, 7.5))
plt.imshow(np.transpose(mag), origin="lower", aspect="auto", cmap='jet', interpolation="none")
plt.colorbar()

plt.xlabel("time (s)")
plt.ylabel("frequency (hz)")

plt.show()

## Loading the data

In [ ]:
# Loading the data before training the neural network

import os
import torch
import numpy as np
import typing


"""
def load_folder(fold):
    features = []
    for fl in os.listdir(basefolder + fold):
        if fl.endswith('.npy'):
            features.append(np.load(basefolder + fold + '/' + fl))
    return np.stack(features)
"""

def load_folder(fold):
    features = []
    for fl in os.listdir(os.path.join(basefolder, fold)):
        if fl.endswith('.npy'):
            features.append(np.load(os.path.join(basefolder, fold, fl)))
    return np.stack(features)

train_features = []
train_labels = []
test_features = [] 
test_labels = []
for i, vowel in enumerate('aeiou'):
    features = load_folder(vowel)
    nexamples = len(features)
    labels = [i] * nexamples
    l = int(0.8 * nexamples)
    train_features.append(features[:l]) 
    train_labels.append(labels[:l])
    test_features.append(features[l:]) 
    test_labels.append(labels[l:])
X_train = np.concatenate(train_features)
y_train = np.concatenate(train_labels)
X_test = np.concatenate(test_features)
y_test = np.concatenate(test_labels)

##### torch
X_train = torch.from_numpy(X_train)
y_train = torch.from_numpy(y_train)
X_test = torch.from_numpy(X_test)
y_test = torch.from_numpy(y_test)
  
print (X_train.shape, y_train.shape, X_test.shape, y_test.shape)

3. Use the variables `X_train`, `y_train`, `X_test` and `y_test` to create datasets (`torch.utils.data.TensorDataset`) and dataloaders. Use a batch_size of 32 for the dataloaders.

4. Have a look at the implementation of the MyRNNLayer, which is used to wrap different types of RNNCells in a recurrent manner. The RNNLayer can be initialised either to output a sequence, i.e., to forward one output for each element of the sequence to the next layer, or to forward only one output at the last element of the sequence to the next layer. This has no effect on the calculation of the hidden state or cell state, but only affects, which values are used as an output. 

In [ ]:
class MyRNNLayer(torch.nn.Module):
    def __init__(
        self, 
        rnn_cell: torch.nn.Module, 
        return_sequence=False
    ):
        r"""RNN Constructor.

        Args:
            rnn_cell: the cell of the rnn defining how inputs are processed 
            return_sequence: flag controlling whether the ouput is a sequence or single elemnt
        """
        super(MyRNNLayer, self).__init__()
        self.return_sequence = return_sequence
        self.rnn_cell = rnn_cell
    
    def forward(
        self,
        x: torch.Tensor
    ) -> torch.Tensor:
        r"""Processes the incoming sequence according to the RNNcell.

        Args:
            x: pytorch tensor containing a sequence features 

        Returns:
            torch.Tensor: output of the RNN, either 2D (no sequence) or 3D (sequence) 
        """
        cell = self.rnn_cell.zero_hidden()
        hidden_states = torch.zeros(x.shape[0], x.shape[1], self.rnn_cell.hidden_features)
        for i in range(x.shape[1]):
            hidden, cell  = self.rnn_cell(x[:,i,:], cell)
            hidden_states[:,i,:] = hidden
        if self.return_sequence:
            return hidden_states
        else:
            return hidden
    


5. Implement the MinimalRNNCell, i.e., the methods `__init__`, initialising all parameters of the cell (you can use the function `torch.nn.init.kaiming_uniform_(self.bias)`), the `zero_hidden` method, which initialises the hidden state of the cell as zero and the `forward` method. The `forward` method processes the transition for a single step in a sequence of intputs. It takes the $i$th input from the previous layer $x_i$ (e.g. input layer) and the hidden state of the previous step $c_{i-1}$, and computes the state and output for the current step $c_i, h_i$. For a simple recurrent network, the output and the state of of a given time step are the same. The transition for a single time step is given by: $c_i = h_i = \text{tanh}(x_i W_x + h_{i-1} W_h + b)$.

In [ ]:
class MinimalRNNCell(torch.nn.Module):
    def __init__(
        self, 
        in_features: int, 
        hidden_features:int,
        device:str
    ):
        r"""Simple RNN Cell Constructor.
        
        Args:
            in_features: number of input features (in each sequence) 
            hidden_features: number of features in the hidden state
        """
        super().__init__()
        
        # TODO Parameter initialisation
                
    def forward(
        self, 
        x: torch.Tensor, 
        c: torch.Tensor
    )-> typing.Tuple[torch.Tensor, torch.Tensor]:
        r"""Processes the incoming input and previous cell state for one step in the sequence.

        Args:
            x: pytorch tensor containing an element of the input sequence
            c: pytorch tensor containing the previous cell state

        Returns:
            (torch.Tensor, torch.Tensor): output to the next layer (vert), and cell state (for next sequence elemnt) 
        """
        
        # TODO Define forward pass for one step of the sequence
        return output, output
        
    def zero_hidden(
        self
    )-> torch.Tensor:
        r"""Initialises cell state with zeros. Assign the hidden state to the model device (necessary when using GPU)

        Returns:
            torch.Tensor: cell state initialised with zeros  
        """

        # TODO Return initial hidden state with zeros, consider the necessary shape (remember broadcasting)!
        return hidden

6. Implement your own RNN model below. The model should consist of one RNN layer with a minimal RNN cell with a hidden layer size of 128, followed by a dense classification layer with 5 outputs according to the 5 classes. Use a `torch.nn.Linear` layer as a classification layer.

In [ ]:
class MyRNN(torch.nn.Module):
    def __init__(
        self, 
        in_size:int, 
        hidden_size:int, 
        output_size:int,
        device:str
    ):
        super().__init__()
        # TODO: Define Cells and Layers 
    
    def forward(
        self, 
        x
    ):
        # TODO: Implement forward
        
        return out

Below are suggested solutions for the implementation of the routine for training and evaluation, and all necessary functions following yesterday's examples. The functions should be ready to run.   

In [ ]:
def single_model_step(
    model: torch.nn.Module, 
    optimizer: torch.optim.Optimizer,
    loss_function: typing.Callable,
    training: bool,
    X: torch.Tensor, 
    y: torch.Tensor,
    device: str
) -> torch.Tensor:
    r"""Single model training/evaluation step.

    Args:
        model: pytorch model to be trained
        optimizer: optimizer wrapping pytorch model
        loss_function: loss function
        training: flag controlling whether this is a training
            or an evaluation step
        X: pytorch tensor containing features
        y: pytorch tensor containing labels
        device: device to where model is located
        
    Returns:
        torch.Tensor: loss at current step
    """
    pred = model(X.to(device))
    loss = loss_function(pred, y.to(device))
    if training:
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    return loss.cpu().item()

def inference_model(
    model: torch.nn.Module, 
    loader: torch.utils.data.DataLoader,
    num_classes,
    device: str
):
    r"""Evaluate model on dataloader.
    
    Iterates over loader until it is exhausted 
    and stores the model output (logits) and labels
    in tensors.
    
    Args:
        model: model to be evaluated
        loader: dataloader containing data to evaluate model on
        device: string denoting device on which to run evaluation
        
    Returns:
        torch.Tensor: model outputs (logits)
        torch.Tensor: ground truth labels
    """
    model.eval()
    logits = torch.zeros((len(loader.dataset), num_classes))
    labels = torch.zeros((len(loader.dataset)))
    batch_size = loader.batch_size
    with torch.no_grad():
        for index, (X, y) in enumerate(loader):
            logits[index*batch_size:(index+1)*batch_size, :] = model(X.to(device)).cpu()
            labels[index*batch_size:(index+1)*batch_size] = y
    return logits, labels

#logits, labels = inference_model(model, test_loader, 5, device)
def get_accuracy(labels, logits):
    r"""Compute accuracy.
    
    Args:
        labels: ground truth labels
        logits: model predictions
    
    Returns:
        torch.Tensor: single accuracy value
    """
    prediction_indices = torch.argmax(logits, axis=-1)
    correct_predictions = labels == prediction_indices
    accuracy = correct_predictions.sum() / len(correct_predictions)
    return accuracy.item()





def train_and_evaluate(
    model: torch.nn.Module, 
    loss_function: typing.Callable, 
    optimizer: torch.optim.Optimizer, 
    train_loader: torch.utils.data.DataLoader,
    test_loader: torch.utils.data.DataLoader,
    num_classes: int,
    epochs: int,
    device: str,
    verbose: bool,
):
    r"""Run training and evaluation.
    
    Args:
        model: pytorch model to be trained
        loss_function: loss function
        optimizer: optimizer wrapping pytorch model
        train_loader: dataloader containing training data
        test_loader: dataloader containing training data
        num_classes: number of classes for the classification task
        epochs: number of epochs for which to train model
        device: device to where model is located
        verbose: flag controlling whether to print information
        
    Returns:
        torch.Tensor: accuracy on test set after last epoch
    """
    for epoch in range(epochs):
        model.train()
        train_loss = 0
        for train_steps, (X, y) in enumerate(train_loader):
            train_loss += single_model_step(
                model=model, 
                loss_function=loss_function, 
                optimizer=optimizer, 
                training=True,
                X=X, 
                y=y,
                device=device
            )
        train_loss /= (train_steps + 1)
        
        model.eval()
        test_loss = 0
        with torch.no_grad():
            for test_steps, (X, y) in enumerate(test_loader):
                test_loss += single_model_step(
                    model=model, 
                    loss_function=loss_function, 
                    optimizer=optimizer, 
                    training=False,
                    X=X, 
                    y=y,
                    device=device
                )
        test_loss /= test_steps
        
        logits, labels = inference_model(
                model=model, 
                loader=test_loader,
                num_classes=num_classes,
                device=device
            )
        accuracy = get_accuracy(
            labels=labels,
            logits=logits
        )
        
        if verbose:
            print(f"\n---------- EPOCH {epoch + 1} ------------")
            print(f"Average Train Loss: {train_loss}")
            print(f"Average Test Loss: {test_loss}")
            
        
            print(f"Test Accuracy: {accuracy}")
    return accuracy

7. Instantiate your own RNN model, then run the train_and_evaluate function implemented above using a cross entropy loss function, an Adam optimiser with `0.001` learning rate for 40 epochs. Finally, create a confusion matrix, a table that lists how often samples of the classes were predicted to be within any of the (other) classes. You can use an `sklearn` implementation. Note: We recommend using the CPU for training in today's exercise sheet as the models are fast to train with CPU as well and you do not risk running out of GPU resources.

In [ ]:
torch.manual_seed = 42
device = 'cpu'

# Instantiate model and run training and evaluation

## Advanced Model Architecture

8. Complete the the `LSTMCell` class below. Now, use this recurrent cell with a hidden layer size of 128 instead of the previous `MinimalRNNCell` for a new RNN class. The `forward` method shall return the new cell output and the new cell state(s). Train your model for 50 epochs with an Adam optimiser and learning rate equal to 0.01. The implementation of `MyRNN2` should be very similar to `myRNN`.

In [ ]:
class LSTMCell(torch.nn.Module):
    def __init__(
        self, 
        in_features: int, 
        hidden_features: int,
        device:str
    ):
        r"""LSTM Cell Constructor.
        
        Args:
            in_features: number of input features (in each sequence) 
            hidden_features: number of features in the hidden and cell state, respectively
        """
        super().__init__()
        # Initialise Parameters
        

    def forward(
        self, 
        x: torch.Tensor, 
        c: (torch.Tensor, torch.Tensor)
    )-> typing.Tuple[torch.Tensor, typing.Tuple[torch.Tensor, torch.Tensor]]:
        r"""Processes the incoming input and previous cell state for one step in the sequence.

        Args:
            x: pytorch tensor containing an element of the input sequence
            c: tuple of pytorch tensors containing the previous hidden and cell state

        Returns:
            (torch.Tensor, (torch.Tensor, torch.Tensor)): \
            output to the next layer (vert), and hidden and cell state (for next sequence elemnt) 
        """
        # Define forward
        pass
        
    def zero_hidden(
        self, 
    )-> typing.Tuple[torch.Tensor, torch.Tensor]:
        r"""Initialises cell state with zeros. Assign the hidden state to the model device (necessary when using GPU)

        Returns:
            (torch.Tensor, torch.Tensor): cell and hidden state initialised with zeros  
        """
        # TODO Return initial hidden state with zeros, consider the necessary shape (remember broadcasting)!
        pass

In [ ]:
class MyRNN2(torch.nn.Module):
    def __init__(
        self, 
        in_size:int, 
        hidden_size:int, 
        output_size:int,
        device:str
    ):
        super().__init__()
        # TODO: Define Cells and Layers 
    
    def forward(
        self, 
        x
    ):
        # TODO: Implement forward
        
        return out

In [ ]:
torch.manual_seed = 42



9. Add more recurrent layers. The second recurrent layer takes as input the output sequence from the first recurrent layer, and so on. That is, make sure each recurrent layer returns the full sequence instead of only the last output. Train your network as in the previous exercise

10. Hyperparameters are all parameters of your model that you define a priori and that will not be tuned/learned during training. Examples are: learning rate, batch size, optimizer, size of the hidden layer or number of hidden layers. Explore the hyperparameter space to find a better performance. You can run for instance a grid search with wisely chosen hyperparameters and hyperparameters. Can you get a test accuracy beyond 85%?